In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio transformers accelerate


In [ ]:
MODEL_DIR = "/content/drive/MyDrive/ibm_granite_2b"


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Use a local directory in Colab for caching the model
MODEL_DIR = "/content/ibm_granite_cache"
os.makedirs(MODEL_DIR, exist_ok=True)

# Model path
model_path = "ibm-granite/granite-3.0-2b-instruct"

# Choose device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir=MODEL_DIR)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    cache_dir=MODEL_DIR,
    device_map="auto" if device == "cuda" else None,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
model.eval()

print("Model loaded successfully on", device)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully on cpu


In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
model_path = "ibm-granite/granite-3.0-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto" if device == "cuda" else None,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
)
model.eval()


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def respond(message, history):
    # Format prompt with role tags for Granite
    prompt = ""
    for turn in history:
        prompt += f"<|start_of_role|>{turn['role']}<|end_of_role|>{turn['content']}<|end_of_text|>\n"
    prompt += f"<|start_of_role|>user<|end_of_role|>{message}<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    reply = generated.strip()

    # Maintain history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": reply})

    return history


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 💰 IBM Granite-2B Finance Chatbot\nAsk me about savings, taxes, or investments!")

    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(placeholder="Type your question here...")

    def user_message(inp, hist):
        return "", respond(inp, hist)

    msg.submit(user_message, [msg, chatbot], [msg, chatbot])

demo.launch(debug=False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f6d66b4ebd5bfc256.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
